1. initialize MCMC parameters
2. intialize the Galaxy object with tilted ring params, baryon rotation curves
3. generate start position mass model -> generate mock 2d velocity field using Galaxy attributes from (2) -> iterate MCMC 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from emcee import EnsembleSampler

import numpy as np
import os
import sys

pwd = os.getcwd()
little_things_root_dir = os.path.dirname(pwd)
sys.path.append(little_things_root_dir)


from little_things_lib.helpers import auto_assign
from little_things_lib.galaxy import Galaxy
from little_things_lib.mcmc_fitter import (
    EmceeParameters,
    get_mcmc_start_position, 
    generate_nwalkers_start_points, 
    lnprob)

class EmceeParameters:
    @auto_assign
    def __init__(
        self,
        ndim,
        nwalkers,
        nburn, 
        niter,
        nthin,
        nthreads
    ):
        pass
    


In [ ]:
mcmc_params = EmceeParameters(
    ndim=4,
    nwalkers=10, 
    nburn=50,
    niter=200,
    nthin=0,
    nthread=4
)

In [2]:
data_dir = os.path.join(little_things_root_dir, 'data')

In [3]:
# all values are just placeholders for testing that functions work
stellar_radii = np.linspace(0, 20, 41)
gas_radii = np.linspace(0, 20, 41)
stellar_vel = np.linspace(0, 20, 41)
gas_vel = np.linspace(0, 20, 41)

mcmc_parameters = EmceeParameters(
    ndim=4,
    nwalkers=10,
    nburn=10,
    niter=50,
    nthin=0,
    nthreads=4)

# set a mock 'observed' 2d velocity field for testing purposes
test_velocity_2d_field = np.load('test_2d_velocity_field_100x100.npy')

galaxy = Galaxy(
    distance_to_galaxy=32,
    deg_per_pixel=4.11e-4,
    image_xdim=100,
    image_ydim=100,
    galaxy_name=None,
    vlos_2d_data=test_velocity_2d_field,
    output_dir='output',
    luminosity=1e8,
    HI_mass=1e6)

galaxy.set_tilted_ring_parameters(
    v_systemic=0,
    radii=np.linspace(0.1, 16, 33),
    inclination=np.full(33, 45),
    position_angle=np.full(33, 15),
    x_pix_center=np.full(33, 50),
    y_pix_center=np.full(33,50))

# TODO: Need to check if the stellar contribution assumes M/L != 1
galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='stellar',
    rotation_curve_radii=stellar_radii,
    rotation_curve_velocities=stellar_vel)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='gas',
    rotation_curve_radii=gas_radii,
    rotation_curve_velocities=gas_vel)

# initialize MCMC start position and bounds
galaxy.set_prior_bounds(
    cross_section_bounds=(2.99, 3.001),
    rate_bounds=(2, 1e4),
    sigma0_bounds=(2, 1e3),
    ml_bounds=(0.1, 10),
    ml_median=0.5,
    rmax_prior=False,
    vmax_prior=False,
    log10_rmax_spread=0.11,
    log10_c200_spread = 0.11,
    abs_err_vel_factor=0.05,
    tophat_width=3)


In [5]:
nwalkers, ndim= 10, 4

start_point, start_point_radii = get_mcmc_start_position(galaxy)
start_pos = generate_nwalkers_start_points(mcmc_params.nwalkers, start_point, start_point_radii)

sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy], 
    mcmc_params.nthreads)
sampler.run_mcmc(start_pos, mcmc_params.niter)


In [ ]:
fits.open(